In [1]:
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
          'width': 1024,
          'height': 768,
#          #'width': 3840,
#          #'height': 2160,    
#          #'width': 1200,
#          #'height': 600,    
})

{'height': 768, 'width': 1024}

# The Hartree Fock method

<div class="col-md-10">
<p> In this tutorial we use a the Python scripting language and an interactive Jupyter notebook to run a Hartree-Fock calculation using the Python-based quantum chemistry code Psi4. </p>


<p> You can install this setup on your own Windows/Mac computer following the instructions on the github mainpage.</p>
</div>
<div class="col-md-2">
<img src="http://www.psicode.org/psi4manual/master/_static/psi4square.png" alt="Psi4 code" style="width: 100px;"/>
</div>

You will see lot's of Python statements that import packages or access functions, such as the following:


Here we load important packages, such as a visualization programm and the Psi4 software package and then we do a little arithmetics. We also define a little helper function. 
You can execute the cell below by either pushing the play putton above or by clicking into the cell and pressing "Shift+Enter". You have to do this for all following cells.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import psi4
from ase.build import molecule
from ase.atoms import Atoms
from ase.visualize import view
from ase.visualize.plot import plot_atoms

print(1+1)

2


In [3]:
def geom_ase_to_psi4(atoms, charge=0, multiplicity=1, symmetry='c1' ):
    """
    takes an ase atoms object and build a Psi4 geometry string
    """
    species = atoms.get_chemical_symbols()
    pos = atoms.get_positions()
    string = "unit angstrom \n"
    string += "symmetry c1 \n"
    string += "{0} {1} \n".format(charge, multiplicity) 
    for i,(spec,p) in enumerate(zip(species, pos)):
        string += "{0} {1:14.8f} {2:14.8f} {3:14.8f}\n".format(spec, p[0], p[1], p[2])
        
    return string

## Part 1: Theory and Method

Hartree-Fock Self-Consistent Field Theory (HF-SCF) with restricted orbitals and closed-shell systems (RHF) solves the so-called Rothaan-Hall equation, which is a generalized eigenvalue matrix equation:

\begin{equation}
\mathbf{FC}=\mathbf{ S C \epsilon}
\end{equation}

The Fock matrix, $\mathbf{F}$, has elements $F_{\mu\nu}$ given as
\begin{equation}
F_{ab} = H_{ab} + \left[ \langle \phi_a|\hat{J}|\phi_b  \rangle - \langle \phi_a|\hat{K}|\phi_b \rangle \right]
\end{equation}
The overlap matrix is defined as
\begin{equation}
S_{ab} = \langle \phi_a|\phi_b \rangle
\end{equation}

The orbital coefficient matrix $\mathbf{C}$ is a $N\times M$ matrix, where $N$ is the number of atomic basis functions and $M$ ($\phi$) is the number of molecular orbitals ($\psi$). This matrix describes the contribution of each basis function $\phi$ to the molecular orbital $\psi$.

## Part 2: A simple Hartree-Fock calculation

### Step 1: Setting up the molecular coordinates
___________

This is the first step of every calculation. We need to define the molecular coordinates. 
The `ase` package contains a lot of handy tools to define molecules with preset geometries.

In the following, we use the function `molecules` from `ase.build` to build a water molecule.

In [4]:
#for water, type H2O
#for ammonia, type NH3
#for di-amino-benzene, type BDA,
#for the buckminster-fullerene, type C60
atoms = molecule('H2O')
print(atoms.positions)

[[ 0.        0.        0.119262]
 [ 0.        0.763239 -0.477047]
 [ 0.       -0.763239 -0.477047]]


Let's visualize the molecule on the screen using `ase.visualize.view`
There are two possible viewers. 'x3d' is the standard one. 'ngl' or nglview has to be installed separately (see handout) but has a lot more features. Try to replace 

`view(atoms, viewer='x3d')` with `view(atoms, viewer='ngl')`

In [11]:
view(atoms,viewer='x3d') #x3d or ngl

### Step 2: Performing the calculation
___________
To calculate the HF energy, we use the Psi4 package. In the following,

* we set the RAM memory, which we allocate to Psi4, 
* we pass the geometry from our `ase.atoms` object to `psi4.geometry`
* we initiate the HF (also called scf) calculation using a split-valence 3-21G* Pople basis set
* we perform the HF energy calculation using `psi4.energy`

In [5]:
psi4.core.clean()
#psi4.core.reopen_outfile()
psi4.core.set_output_file('output.dat', False)
psi4.set_memory('1000 MB')

##set geometry
#transform ase geometries to Psi4 input string
geom_input = geom_ase_to_psi4(atoms, charge=0, multiplicity=1)
#initiate Psi4 molecule object
nh3 = psi4.geometry(geom_input)

# Set computation options
psi4.set_options({'basis': '3-21G',
                  'reference': 'rhf',
                  'scf_type':'direct',
                  'e_convergence': 1e-8})

E, wvf = psi4.energy('scf',return_wfn=True)
print('Total HF energy is {0} Hartree \n '.format(E))


Total HF energy is -75.58555599788606 Hartree 
 


We have successfully calculated the HF energy and wave function. Let's take a closer look at the detailed output statements written in `output.dat`.

In [6]:
%less output.dat

We can also analyze the results further by looking at the MO energies, the occupations, and the coefficients.

In [7]:
#Let's look at orbital coefficients, MO energies and occupations
coeffs = wvf.Ca().to_array()
epsilons = wvf.epsilon_a().to_array()
occs = wvf.occupation_a().to_array()
for i,(o,e) in enumerate(zip(occs, epsilons)):
    print ('Molecular Orbital {0}, occupation: {1:4.2f}, energy: {2:8.3f}'
           .format(i,o*2,e))
print('\n The wave function matrix has dimensions:', coeffs.shape)
print('\n Wave function coefficients for the HOMO orbital:')
for i in coeffs[:,4]:
    print('{0:6.2f}'.format(i))    

Molecular Orbital 0, occupation: 2.00, energy:  -20.431
Molecular Orbital 1, occupation: 2.00, energy:   -1.324
Molecular Orbital 2, occupation: 2.00, energy:   -0.679
Molecular Orbital 3, occupation: 2.00, energy:   -0.537
Molecular Orbital 4, occupation: 2.00, energy:   -0.479
Molecular Orbital 5, occupation: 0.00, energy:    0.260
Molecular Orbital 6, occupation: 0.00, energy:    0.358
Molecular Orbital 7, occupation: 0.00, energy:    1.185
Molecular Orbital 8, occupation: 0.00, energy:    1.297
Molecular Orbital 9, occupation: 0.00, energy:    1.782
Molecular Orbital 10, occupation: 0.00, energy:    1.871
Molecular Orbital 11, occupation: 0.00, energy:    2.015
Molecular Orbital 12, occupation: 0.00, energy:    3.108

 The wave function matrix has dimensions: (13, 13)

 Wave function coefficients for the HOMO orbital:
  0.00
 -0.00
  0.52
  0.00
 -0.00
 -0.00
  0.63
  0.00
 -0.00
  0.00
  0.00
  0.00
  0.00


From the above, we learn that the molecule has 13 AO basis functions, which define 13 Molecular Orbitals. Of these 13 orbitals, 5 are occupied. (These are spatial orbitals, each of which has 2 electrons. That makes 10 electrons in H$_2$O).

The coefficients of the HOMO orbital tell us that it is only comprised of 2 different AOs. 

The last function here prints a so-called molden file, with which we can visualize the molecular orbitals in Avogadro.
We can use [Avogadro](https://avogadro.cc/) to figure out which ones.

In [8]:
#Write wave functions to molden file format, can be visualized in Avogadro
psi4.molden(wvf, 'h2o.molden')

## Want to learn more about the inner workings of Hartree Fock?

Check out the psi4numpy tutorial on Hartree-Fock

https://github.com/psi4/psi4numpy/blob/master/Tutorials/03_Hartree-Fock/3a_restricted-hartree-fock.ipynb